In [1]:
import os
from ultralytics import NAS
os.chdir("..")
ROOT = os.getcwd()
print(ROOT)

/Users/mautushid/github/Cowsformer


In [2]:
from models.nas import *
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np
import supervision as sv
from models.nas import Niche_YOLO_NAS
from super_gradients.training.models import get as get_model
from super_gradients.training import Trainer
from super_gradients.training import models
import torch
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback



The console stream is logged into /Users/mautushid/sg_logs/console.log


[2023-12-06 13:43:04] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-12-06 13:43:05] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-12-06 13:43:05] WARNING - redirects.py - NOTE: Redirects are currently not supported in Windows or MacOs.
[2023-12-06 13:43:15] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-12-06 13:43:15] WARNING - export.py - Failed to import pytorch_quantization
[2023-12-06 13:43:15] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-12-06 13:43:15] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernels. Some features may not work as expected.


In [3]:
dataset = sv.DetectionDataset.from_yolo(images_directory_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test/images",
    annotations_directory_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test/labels",
    data_yaml_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/data.yaml",
    force_masks=False)

In [4]:
path_model = 'yolo_nas_m'
data_yaml_path = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/data.yaml"
dir_train = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/train"
dir_val = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/val"
dir_test = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test"
name_task = "cow200"
with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)

CLASSES = data.get('names', [])

print("classes:", CLASSES)

classes: ['cows']


In [5]:
my_nas = Niche_YOLO_NAS(path_model, dir_train, dir_val, dir_test, name_task)
finetuned_model_path_m = "/Users/mautushid/github/Cowsformer/lms_checkpoints/n10_yolo_na_i1_exp_yolo_nas_m_10_1/ckpt_best.pth"
model = my_nas.load(finetuned_model_path_m)

[2023-12-06 13:43:23] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-12-06 13:43:23] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_m


model /Users/mautushid/github/Cowsformer/lms_checkpoints/n10_yolo_na_i1_exp_yolo_nas_m_10_1/ckpt_best.pth loaded


In [6]:
dataset_params = {
    'data_dir': '/Users/mautushid/github/Cowsformer/data/cow200/yolov5',
    'train_images_dir': 'train/images',
    'train_labels_dir': 'train/labels',
    'val_images_dir': 'val/images',
    'val_labels_dir': 'val/labels',
    'test_images_dir': 'test/images',
    'test_labels_dir': 'test/labels',
    'classes': CLASSES
}
########################################
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)


[2023-12-06 13:43:25] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 221/221 [00:01<00:00, 195.83it/s]
[2023-12-06 13:43:26] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 35/35 [00:00<00:00, 288.80it/s]
[2023-12-06 13:43:26] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.


Indexing dataset annotations: 100%|██████████| 100/100 [00:00<00:00, 2220.10it/s]


In [7]:
train_params = {
    'silent_mode': False,
    "average_best_models": True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": 10,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}


In [8]:
model

YoloNAS_L(
  (backbone): NStageBackbone(
    (stem): YoloNASStem(
      (conv): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        )
        (branch_1x1): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2))
        (post_bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (rbr_reparam): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
    )
    (stage1): YoloNASStage(
      (downsample): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=Tr

In [ ]:
import supervision as sv
from ultralytics import YOLO

# Assuming you have a function to load your dataset
# Each item in the dataset should be a tuple (image, ground_truth)
dataset = load_your_dataset()  # Replace with your dataset loading method

# Initialize your model
model = YOLO(...)  # Replace with your model initialization

# Lists to store targets and predictions
targets = []
predictions = []

# Iterate over the dataset
for image, ground_truth in dataset:
    # Generate prediction for the image
    prediction = model(image)[0]  # Adjust this line according to your model's output format

    # Convert ground truth and prediction to sv.Detections format
    # Note: You will need to replace 'convert_to_sv_detections' with the actual conversion method
    ground_truth_sv = convert_to_sv_detections(ground_truth)
    prediction_sv = convert_to_sv_detections(prediction)

    # Append to lists
    targets.append(ground_truth_sv)
    predictions.append(prediction_sv)

# Calculate Mean Average Precision
mean_average_precision = sv.MeanAveragePrecision.from_detections(
    predictions=predictions,
    targets=targets,
)

# Output mAP
print(mean_average_precision.map50_95)
